### Import libraries and define vars

In [1]:
# import all libraries
from music21 import *
from keras.layers import *
import matplotlib.pyplot as plt
import numpy as np
import os
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from collections import Counter
import keras.backend as K
from keras.callbacks import *
import random

In [2]:
# define variables and hyperparameters
EPOCHS = 100
BATCH_SIZE = 256
DATA_PATH = "data/"

### Load data

In [3]:
def read_midi(file):
    """
    This function will read midi files and return all of it notes
    """
    # define list for notes and variable for notes to parse
    notes = list()
    notes_to_parse = None

    # read midi files
    midi = converter.parse(file)

    # Seperate instruments
    instruments = instrument.partitionByInstrument(midi)

    # loop over all instruments
    for part in instruments.parts:
        # select only piano notes
        if "Piano" in str(part):
            notes_to_parse = part.recurse()

            # find if the element is chord or single note
            for element in notes_to_parse:
                # to select notes
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))

                # to select chords
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))


    # put notes into an array
    notes_array = np.array(notes)

    return notes_array

In [4]:
# read all of the file names
files=[i for i in os.listdir(DATA_PATH) if i.endswith(".mid")]

# load the musics
notes_array = np.array([read_midi(DATA_PATH + i) for i in files])

C:\Users\yasin\AppData\Local\Temp\ipykernel_2424\2770618029.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  notes_array = np.array([read_midi(DATA_PATH + i) for i in files])


### Preprocess data

In [5]:
# converting 2D array into 1D array
notes_1d = [element for note_ in notes_array for element in note_]

# get number if unique notes
unique_notes = list(set(notes_1d))

print(f"We have {len(unique_notes)} unique notes.")

We have 1226 unique notes.


In [6]:
# see how many frequent notes we have 
frequent = dict(Counter(notes_1d))
frequent_notes = [note_ for note_, count in frequent.items() if count>=50]

print(f"We have {len(frequent_notes)} frequent notes.")

We have 383 frequent notes.


In [7]:
# define list for new music
new_music = list()

# Add all frequent notes into new music
for notes in notes_array:
    temp = list()
    for note_ in notes:
        if note_ in frequent_notes:
            temp.append(note_)

    new_music.append(temp)

# convert new music into numpy array
new_music = np.array(new_music)

C:\Users\yasin\AppData\Local\Temp\ipykernel_2424\2819184097.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  new_music = np.array(new_music)


In [8]:
# define variable for X, y and number of timesteps
n_timesteps = 32
X = list()
y = list()

# loop over all notes in new music
for notes in new_music:
    for i in range(0, len(notes) - n_timesteps, 1):

        # prepare input and output
        input_data = notes[i:i + n_timesteps]
        output_data = notes[i + n_timesteps]

        # add input and output data in to X and y lists
        X.append(input_data)
        y.append(output_data)

In [9]:
# convert X and y into numpy arrays
X = np.array(X)
y = np.array(y)

In [10]:
# assign int numbers into each frequent note
unique_x = list(set(X.ravel()))
x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

In [11]:
# prepare input seauence 
x_sequence = list()

for i in X:
    temp = list()
    for j in i:
        temp.append(x_note_to_int[j])
    
    x_sequence.append(temp)

x_sequence = np.array(x_sequence)

In [12]:
# preprare output data
unique_y = list(set(y))
y_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_y)) 
y_sequence = np.array([y_note_to_int[i] for i in y])

In [13]:
# split data into train and test 
train_input, val_input, train_output, val_output = train_test_split(x_sequence,
                                                                    y_sequence,
                                                                    test_size=0.2,
                                                                    random_state=0)

In [14]:
# convert data into numpy arrays
train_input = np.array(train_input)
train_output = np.array(train_output)
val_input = np.array(val_input)
val_output = np.array(val_output)

### Neural Networks

In [16]:
# define input and output shapes
input_shape = len(unique_x)
output_shape = len(unique_y)

In [376]:
K.clear_session()

model = Sequential()
    
# input layer
model.add(Embedding(input_shape, 100, input_length=32, trainable=True)) 

# Conv block 1
model.add(Conv1D(64, 3, padding='causal', activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
    
# Conv block 2
model.add(Conv1D(128, 3, dilation_rate=2, padding='causal', activation='relu',))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))

# Conv block 3
model.add(Conv1D(256, 3, dilation_rate=4,padding='causal', activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))

# Global maxPooling
model.add(GlobalMaxPool1D())
    
# Dense layers
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation="relu"))

# Output layer
model.add(Dense(output_shape, activation='softmax'))

In [377]:
# compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

In [378]:
# see the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 100)           38300     
                                                                 
 conv1d (Conv1D)             (None, 32, 64)            19264     
                                                                 
 dropout (Dropout)           (None, 32, 64)            0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 16, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 16, 128)           24704     
                                                                 
 dropout_1 (Dropout)         (None, 16, 128)           0         
                                                        

In [379]:
call_back = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True,verbose=1)

In [380]:
# train the model
history = model.fit(train_input,
                    train_output,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(val_input, val_output),
                    verbose=1,
                    callbacks=[call_back])

Epoch 1/100
2667/2671 [============================>.] - ETA: 0s - loss: 4.2674
Epoch 1: val_loss improved from inf to 4.12488, saving model to best_model.h5
2671/2671 [==============================] - 22s 8ms/step - loss: 4.2670 - val_loss: 4.1249
Epoch 2/100
2671/2671 [==============================] - ETA: 0s - loss: 3.9161
Epoch 2: val_loss improved from 4.12488 to 3.93774, saving model to best_model.h5
2671/2671 [==============================] - 20s 8ms/step - loss: 3.9161 - val_loss: 3.9377
Epoch 3/100
2663/2671 [============================>.] - ETA: 0s - loss: 3.8131
Epoch 3: val_loss improved from 3.93774 to 3.84919, saving model to best_model.h5
2671/2671 [==============================] - 20s 8ms/step - loss: 3.8129 - val_loss: 3.8492
Epoch 4/100
2670/2671 [============================>.] - ETA: 0s - loss: 3.7557
Epoch 4: val_loss improved from 3.84919 to 3.80118, saving model to best_model.h5
2671/2671 [==============================] - 21s 8ms/step - loss: 3.7556 - val_l

2667/2671 [============================>.] - ETA: 0s - loss: 3.5125
Epoch 34: val_loss improved from 3.56050 to 3.55144, saving model to best_model.h5
2671/2671 [==============================] - 20s 7ms/step - loss: 3.5124 - val_loss: 3.5514
Epoch 35/100
2669/2671 [============================>.] - ETA: 0s - loss: 3.5103
Epoch 35: val_loss did not improve from 3.55144
2671/2671 [==============================] - 21s 8ms/step - loss: 3.5104 - val_loss: 3.5565
Epoch 36/100
2668/2671 [============================>.] - ETA: 0s - loss: 3.5074
Epoch 36: val_loss did not improve from 3.55144
2671/2671 [==============================] - 21s 8ms/step - loss: 3.5075 - val_loss: 3.5579
Epoch 37/100
2664/2671 [============================>.] - ETA: 0s - loss: 3.5084
Epoch 37: val_loss did not improve from 3.55144
2671/2671 [==============================] - 21s 8ms/step - loss: 3.5085 - val_loss: 3.5563
Epoch 38/100
2668/2671 [============================>.] - ETA: 0s - loss: 3.5062
Epoch 38: val

2669/2671 [============================>.] - ETA: 0s - loss: 3.4823
Epoch 70: val_loss did not improve from 3.53366
2671/2671 [==============================] - 20s 7ms/step - loss: 3.4824 - val_loss: 3.5516
Epoch 71/100
2666/2671 [============================>.] - ETA: 0s - loss: 3.4824
Epoch 71: val_loss did not improve from 3.53366
2671/2671 [==============================] - 20s 7ms/step - loss: 3.4825 - val_loss: 3.5365
Epoch 72/100
2669/2671 [============================>.] - ETA: 0s - loss: 3.4793
Epoch 72: val_loss did not improve from 3.53366
2671/2671 [==============================] - 21s 8ms/step - loss: 3.4794 - val_loss: 3.5537
Epoch 73/100
2667/2671 [============================>.] - ETA: 0s - loss: 3.4793
Epoch 73: val_loss did not improve from 3.53366
2671/2671 [==============================] - 21s 8ms/step - loss: 3.4794 - val_loss: 3.5461
Epoch 74/100
2670/2671 [============================>.] - ETA: 0s - loss: 3.4784
Epoch 74: val_loss did not improve from 3.53366


### Load model and make predictions

In [64]:
# load the model
model = load_model('best_model.h5')

In [68]:
# define a list for prediction
prediction_list = list()

# define index
index = np.random.randint(0, len(train_input) - 1)
# define random music
random_music = train_input[index]

for i in range(10):
    random_music = random_music.reshape(1, 32)
    # make prediction
    prediction = model.predict(random_music)[0]
    y_prediction = np.argmax(prediction, axis=0)
    prediction_list.append(y_prediction)
    
    random_music = np.insert(random_music[0],len(random_music[0]),y_prediction)
    random_music = random_music[1:]

1/1 [==============================] - 0s 23ms/step


In [69]:
random_music

array([218, 158, 247, 158,  51, 374, 247, 374, 289, 374, 164, 152, 247,
       152, 249, 337, 218, 158, 247, 158,  51, 374,  67, 374, 374, 374,
       374, 374, 374, 374, 374, 374])

In [70]:
print(prediction_list)

[67, 374, 374, 374, 374, 374, 374, 374, 374, 374]


In [993]:
# convert prediction to notes'
x_int_to_note = dict((number, note_) for number, note_ in enumerate(unique_x)) 
predicted_notes = [x_int_to_note[i] for i in prediction_list]

### Convert prediction to midi

In [994]:
def convert_to_midi(prediction_output, music_name):
    """
    This function will convert notes into midi file
    """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                
                cn=int(current_note)
                new_note = note.Note(cn)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
                
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            
        # pattern is a note
        else:
            
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 1

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=music_name)

In [995]:
convert_to_midi(predicted_notes, 'music_8.mid')